#### Exploring Which Dataset to Use

In [1]:
import pandas as pd
import os

os.chdir('C:/Users/dalto/OneDrive/Pictures/Documents/Projects/Coding Projects/woba modeling/data/')

#### Load Data

In [2]:
cubic_woba = pd.read_csv('sim_results/wobacon_23_25.csv')
woba = pd.read_csv('pitch/pitch_cleaned.csv')
wobacon = pd.read_csv('pitch/xwobacon.csv')

#### Correlations

In [3]:
woba = woba[woba['description'] == 'hit_into_play']
woba = woba.groupby(['batter', 'year'])['woba_value'].mean()
woba = woba.reset_index()

In [5]:
merged = cubic_woba
merged = merged.merge(woba, left_on=['name', 'year'], right_on=['batter', 'year']).drop(columns=['Unnamed: 0'])
merged = merged.merge(wobacon[['name', 'year', 'wobacon', 'xwobacon']], on=['name', 'year'], how='left')

stability

In [8]:
stats = ['ev_dir', 'bat_tracking','wobacon', 'xwobacon']
corr_next = merged.copy()

for stat in stats:
    corr_next[f'{stat}_next'] = corr_next.groupby('name')[stat].shift(-1)

corr_next = corr_next.dropna(subset=[f'{stat}_next' for stat in stats])

corrs = {}
for stat in stats:
    corrs[stat] = corr_next[[stat, f'{stat}_next']].corr().iloc[0, 1]

print(corrs)

{'ev_dir': 0.4462042213422614, 'bat_tracking': 0.693556253216255, 'wobacon': 0.33079515401444803, 'xwobacon': 0.5354507754157568}


dataset to next year wobacon

In [9]:
stats = ['ev_dir', 'bat_tracking','wobacon', 'xwobacon']
prev_stat_next_woba_corr = {}
for stat in stats:
    prev_stat_next_woba_corr[stat] = corr_next[[stat, 'wobacon_next']].corr().iloc[0, 1]

print(prev_stat_next_woba_corr)

{'ev_dir': 0.35933814206948406, 'bat_tracking': 0.4294875807608976, 'wobacon': 0.33079515401444803, 'xwobacon': 0.39818068115771976}


2024 stat to 2025 wobacon

In [10]:
stats = ['ev_dir', 'bat_tracking','wobacon', 'xwobacon']
stat_2024_to_2025_woba_corr = {}
df_2024 = corr_next[corr_next['year'] == 2024]
for stat in stats:
    stat_2024_to_2025_woba_corr[stat] = df_2024[[stat, 'wobacon_next']].corr().iloc[0, 1]

print(stat_2024_to_2025_woba_corr)

{'ev_dir': 0.35299366623190487, 'bat_tracking': 0.4246026012677284, 'wobacon': 0.3379877382103016, 'xwobacon': 0.3746369132171138}
